#  Anàlisi exploratòria dels conjunts de dades d'ADS-B i ICA

Aquest notebook presenta una anàlisi exploratòria inicial de dos conjunts de dades
procedents de fonts diferents però relacionats amb l'objectiu del treball:

- **ADS-B (Automatic Dependent Surveillance–Broadcast)**: Dades de vols individuals
  que contenen informació sobre posició, altitud, velocitat i identificadors
  d'aeronaus, utilitzades com a indicador d'activitat aèria.

- **ICA (Índex de Qualitat de l'Aire)**: Dades ambientals que mesuren la qualitat
  de l'aire a partir de diferents contaminants atmosfèrics.

### Objectiu
L'objectiu d'aquest notebook és comprendre l'estructura, qualitat i característiques
principals de cadascun dels datasets de manera independent, amb la finalitat de:
- Detectar problemes de qualitat de les dades.
- Identificar variables rellevants.
- Establir criteris de neteja i transformació.

Aquesta anàlisi servirà com a base per a una posterior preparació de les dades i,
eventualment, per a l'estudi de possibles relacions entre l'activitat aèria i la
qualitat de l'aire.

### Abast
En aquesta fase es realitza exclusivament una **anàlisi exploratòria descriptiva**,
incloent:
- Dimensions dels datasets.
- Tipus de dades i variables disponibles.
- Presència de valors nuls o en blanc.
- Estadístiques descriptives bàsiques.
- Detecció de duplicats.

No s'aplica cap procés de neteja, filtratge ni integració de dades en aquest notebook.

## 1. Carregar dades brutes

In [1]:
# Importar les biblioteques necessàries
from pathlib import Path
import pandas as pd
import numpy as np

/tmp/ipykernel_21/1028428549.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# Llegir els fitxers CSV
adsb_path = Path("../data/raw/adsb/adsb_raw.csv")
df_adsb = pd.read_csv(adsb_path)

ica_path = Path("../data/merged/ica_merged_raw.csv")
df_ica = pd.read_csv(ica_path)

## 2. Anàlisi Exploratòria de Dades (EDA)

Abans de procedir a qualsevol transformació, s'executa una funció d'anàlisi exhaustiva per comprendre l'estructura, la qualitat i la naturalesa dels conjunts de dades originals:

- Dimensió i Tipologia: Es defineix el volum total de registres i s'identifiquen els tipus de dades de cada columna (numèrics, objectes o temporals). Això permet assegurar que cada variable té el format correcte per a l'anàlisi posterior.

- Diagnòstic de Qualitat (Nuls i Blancs): Es quantifica la presència de valors nuls i de cadenes de text buides. Aquest pas és crucial per decidir quines columnes requereixen una neteja profunda o quines s'han de descartar per falta d'informació rellevant.

- Estadística Descriptiva: Per a les variables numèriques (com l'altitud o l'índex ICA), s'obtenen els valors mínims, màxims, mitjanes i desviacions estàndard. Això ajuda a detectar d'entrada possibles valors atípics (outliers) o errors en els sensors.

- Integritat i Unicitat: Es comptabilitzen els valors únics en les columnes categòriques i es detecten possibles registres duplicats. Aquest control garanteix la traçabilitat i evita que informació repetida pugui esbiaixar els resultats de l'estudi.

In [5]:
# Funció per executar l'anàlisi exploratòria per cada fitxer
def analisi_exploratori(df, nom_fitxer):
    print("="*80)
    print(f"ANÀLISI EXPLORATORI {nom_fitxer}")
    print("="*80)

    # Dimensió del dataset
    print(f"\nDIMENSIÓ DEL DATASET:")
    print(f"Nombre de registres (files): {df.shape[0]}")
    print(f"Nombre de columnes: {df.shape[1]}")

    # Informació de les columnes
    print(f"\nCOLUMNES I TIPUS DE DADES:")
    print(df.dtypes)

    # Valors nuls
    print(f"\nVALORS NULS:")
    nuls = df.isnull().sum()
    percentatge_nuls = (df.isnull().sum() / len(df)) * 100
    nuls_df = pd.DataFrame({
        'Columna': nuls.index,
        'Valors Nuls': nuls.values,
        'Percentatge (%)': percentatge_nuls.values
    })
    nuls_df = nuls_df[nuls_df['Valors Nuls'] > 0].sort_values('Valors Nuls', ascending=False)

    if len(nuls_df) > 0:
        print(nuls_df.to_string(index=False))
    else:
        print("No hi ha valors nuls")

    # Valors en blanc (strings buits o només espais)
    print(f"\nVALORS EN BLANC:")
    blancs = {}
    for col in df.select_dtypes(include=['object']).columns:
        # Comptar strings buits o només amb espais
        count_blancs = df[col].astype(str).str.strip().eq('').sum()
        if count_blancs > 0:
            blancs[col] = count_blancs

    if blancs:
        for col, count in sorted(blancs.items(), key=lambda x: x[1], reverse=True):
            percentatge = (count / len(df)) * 100
            print(f"{col}: {count} ({percentatge:.2f}%)")
    else:
        print("No hi ha valors en blanc")

    # Primeres 5 files
    print(f"\nPRIMERES 5 FILES:")
    print(df.head())

    # Estadístiques descriptives per columnes numèriques
    print(f"\nESTADÍSTIQUES DESCRIPTIVES (columnes numèriques):")
    if len(df.select_dtypes(include=[np.number]).columns) > 0:
        print(df.describe())
    else:
        print("No hi ha columnes numèriques")

    # Valors únics per columnes categòriques/text
    print(f"\nVALORS ÚNICS (columnes de text):")
    for col in df.select_dtypes(include=['object']).columns:
        n_unics = df[col].nunique()
        print(f"{col}: {n_unics} valors únics")

    # 8. Duplicats
    print(f"\nREGISTRES DUPLICATS:")
    duplicats = df.duplicated().sum()
    print(f"Nombre de files duplicades: {duplicats}")
    if duplicats > 0:
        print(f"Percentatge: {(duplicats/len(df)*100):.2f}%")

    print("\n" + "="*80 + "\n")

##  3. Resultats de l'anàlisi exploratòria

In [6]:
# Resultats de l'anàlisi exploratòria pel fitxer d'ADS-B (dades de vols)
analisi_exploratori(df_adsb, "ADS-B (dades de vols)")

ANÀLISI EXPLORATORI ADS-B (dades de vols)

DIMENSIÓ DEL DATASET:
Nombre de registres (files): 137574
Nombre de columnes: 13

COLUMNES I TIPUS DE DADES:
hex           object
r             object
t             object
flight        object
lat          float64
lon          float64
alt_baro       int64
alt_geom     float64
gs           float64
track        float64
type          object
category      object
timestamp     object
dtype: object

VALORS NULS:
 Columna  Valors Nuls  Percentatge (%)
alt_geom         2078         1.510460
category         1085         0.788666
      gs          552         0.401239
       r          358         0.260224
       t          358         0.260224
  flight          336         0.244232
   track          146         0.106125

VALORS EN BLANC:
No hi ha valors en blanc

PRIMERES 5 FILES:
      hex       r     t   flight        lat        lon  alt_baro  alt_geom  \
0  ab048d  N809NW  A333   DAL230  46.033298 -13.166700     37000       NaN   
1  ab5c12  N831AA

L'anàlisi exploratòria del dataset ADS-B mostra un conjunt de dades amb 137.574 registres i 13 variables. El volum de dades és adequat per a anàlisis estadístiques i agregacions posteriors.

**Estructura i tipus de dades:**

El dataset combina:

- Variables identificatives i categòriques (hex, flight, type, category).

- Variables espacials (lat, lon).

- Variables de moviment i vol (alt_baro, alt_geom, gs, track).

- Variable temporal (timestamp).

Les coordenades geogràfiques (lat, lon) i l'altitud (alt_baro, alt_geom) presenten tipus numèrics coherents, mentre que la variable *timestamp* es troba inicialment en format text, fet que requerirà una conversió posterior a format temporal.

**Qualitat de les dades:**

No s'han detectat valors en blanc en cap columna, la qual cosa indica una bona qualitat en la captura textual de les dades.

Pel que fa als valors nuls, aquests es concentren principalment en:

- alt_geom (1,51%).

- category (0,79%).

- gs (0,40%).

Els percentatges de valors nuls són relativament baixos i no comprometen la integritat global del dataset. Tanmateix, caldrà decidir si aquests valors es poden imputar, eliminar o simplement ignorar.

**Distribució i rangs de les variables numèriques:**

Les estadístiques descriptives mostren que la major part dels vols es troben a altituds de creuer:

- Altitud baromètrica mitjana al voltant dels 33.000–36.000 peus.

- Velocitats mitjanes (gs) pròximes als 430–440 nusos.

Els rangs de latitud i longitud indiquen una concentració de vols en una àrea geogràfica coherent amb l'espai aeri europeu, amb alguns extrems que podrien correspondre a vols en trànsit.

**Variables categòriques:**

El nombre elevat de valors únics en *hex* (3.771) i *flight* (9.931) confirma que el dataset inclou múltiples aeronaus i vols diferents. La variable *type* presenta només 5 categories, mentre que *category* en presenta 10, fet que facilita una possible classificació del tipus d’aeronau o del sistema de transmissió.

**Duplicats:**

No s'han detectat registres duplicats, fet que indica que cada observació correspon a un missatge ADS-B únic dins del conjunt de dades.

In [7]:
# Resultats de l'anàlisi exploratòria pel fitxer d'ICA (qualitat de l'aire)
analisi_exploratori(df_ica, "ICA (qualitat de l'aire)")

ANÀLISI EXPLORATORI ICA (qualitat de l'aire)

DIMENSIÓ DEL DATASET:
Nombre de registres (files): 16195
Nombre de columnes: 9

COLUMNES I TIPUS DE DADES:
cod_estacion      int64
nombre           object
tipo             object
latitud         float64
longitud        float64
activa             bool
fecha            object
indice          float64
debido_a         object
dtype: object

VALORS NULS:
 Columna  Valors Nuls  Percentatge (%)
debido_a         2316        14.300710
  indice         1670        10.311825

VALORS EN BLANC:
No hi ha valors en blanc

PRIMERES 5 FILES:
   cod_estacion           nombre     tipo   latitud  longitud  activa  \
0       1022001         EL CIEGO  TRAFICO  42.51833  -2.61944    True   
1       1036004           LLODIO  TRAFICO  43.14407  -2.96337    True   
2       1051001          AGURAIN    FONDO  42.84900  -2.39370    True   
3       1055001        VALDEREJO    FONDO  42.87520  -3.23170    True   
4       1059008  AVENIDA GASTEIZ  TRAFICO  42.85480  -2.680

L'anàlisi exploratòria del dataset corresponent a l'Índex de Qualitat de l'Aire (ICA) mostra un conjunt de dades de 16.195 registres i 9 variables, amb informació ambiental procedent de diferents estacions de mesura distribuïdes territorialment.

Tot i tenir una mida inferior al dataset ADS-B, el volum de dades és suficient per a l'anàlisi temporal i espacial de la qualitat de l'aire, especialment quan s'agrupa per estació, data o contaminant dominant.

**Estructura i tipus de dades:**

El dataset inclou:

- Variables identificatives de l'estació (cod_estacion, nombre, tipo, activa).

- Variables espacials (latitud, longitud).

- Variable temporal (fecha).

- Indicadors ambientals (indice, debido_a).

La presència de coordenades geogràfiques permet la representació espacial de la qualitat de l'aire i la possible agregació per zones. La variable *fecha* es troba en format text, fet que requerirà conversió a format temporal per facilitar l'anàlisi cronològica.

**Qualitat de les dades:**

El dataset no presenta valors en blanc, fet que indica una correcta estructura textual de les dades. Tanmateix, sí que s'observa una proporció rellevant de valors nuls en dues variables clau:

- debido_a: 14,3% de valors nuls.

- indice: 10,3% de valors nuls.

Aquests valors nuls poden indicar absència de mesura en determinats moments, estacions sense càlcul d'índex per aquell període o situacions en què no es pot atribuir l'índex a un contaminant concret. Aquest aspecte requerirà una decisió explícita durant la fase de neteja, ja que *indice* és una variable fonamental per a l'anàlisi posterior.

**Distribució i rangs de les variables numèriques:**

Les estadístiques descriptives mostren:

- Una concentració majoritària d'índexs baixos (mediana = 2), indicant nivells generalment bons o moderats de qualitat de l'aire.

- Un valor màxim d'índex igual a 30, que correspon a episodis puntuals de mala qualitat de l'aire.

- Una distribució geogràfica coherent amb el territori analitzat, amb latituds i longituds pròpies de l'àmbit estatal.

La dispersió de la variable *indice* suggereix que existeixen diferències significatives entre estacions i moments temporals, fet rellevant per a l'anàlisi de patrons espacials i temporals.

**Variables categòriques:**

Les variables categòriques presenten una diversitat moderada:

- tipo: només 3 categories.

- debido_a: 5 contaminants diferents.

- fecha: 26 valors únics, indicant una cobertura temporal discreta.

Aquesta estructura facilita l'agrupació per tipus d'estació i contaminant dominant.

**Duplicats:**

No s'han detectat registres duplicats, fet que indica que cada observació correspon a una combinació única d'estació i moment temporal.

In [8]:
# Comparació entre els dos datasets
print("="*80)
print("COMPARACIÓ ENTRE DATASETS")
print("="*80)
print(f"\nColumnes en comú: {set(df_adsb.columns) & set(df_ica.columns)}")
print(f"Columnes només a fitxer ADS-B: {set(df_adsb.columns) - set(df_ica.columns)}")
print(f"Columnes només a fitxer ICA: {set(df_ica.columns) - set(df_adsb.columns)}")

COMPARACIÓ ENTRE DATASETS

Columnes en comú: set()
Columnes només a fitxer ADS-B: {'alt_baro', 'alt_geom', 'track', 'gs', 'flight', 'type', 'lon', 'lat', 'r', 'timestamp', 'hex', 't', 'category'}
Columnes només a fitxer ICA: {'cod_estacion', 'nombre', 'indice', 'fecha', 'longitud', 'latitud', 'activa', 'tipo', 'debido_a'}


La comparació entre els datasets de vols (ADS-B) i qualitat de l'aire (ICA) mostra que no existeixen columnes en comú entre ambdós conjunts de dades.

**Resultat de la comparació:**

- Columnes en comú: cap.

- Columnes exclusives del dataset ADS-B: variables relacionades amb el trànsit aeri, identificació d'aeronaus, posició, altitud, velocitat i temps (lat, lon, alt_baro, alt_geom, gs, track, timestamp, etc.).

- Columnes exclusives del dataset ICA: variables pròpies de la mesura ambiental, identificació d'estacions, contaminant dominant, índex de qualitat de l'aire i data (cod_estacion, latitud, longitud, indice, fecha, etc.).

Aquesta absència de columnes comunes indica que no és possible una unió directa (join) entre els datasets basada en una clau compartida.

**Implicacions metodològiques:**

El fet que no hi hagi cap camp comú té diverses implicacions importants:

- La relació entre els datasets s'haurà d'establir mitjançant la proximitat espacial (coordenades geogràfiques dels vols i de les estacions), o la coincidència temporal (timestamps dels vols i dates/hores de les mesures ICA).

- Abans de qualsevol anàlisi conjunta serà necessari unificar formats temporals (timestamp vs fecha), i harmonitzar sistemes de coordenades (lat/lon vs latitud/longitud).

- Aquesta comparació justifica clarament l'existència d'una fase específica de neteja, transformació i agregació espacial i temporal, que permeti generar variables derivades compatibles entre ambdós datasets.

### Conclusions preliminars

El conjunt de dades ADS-B presenta una estructura coherent, una qualitat globalment bona i un nivell baix de valors nuls. Aquest conjunt de dades pot utilitzar-se com a indicador de nivell d'activitat aèria, que posteriorment podrà ser relacionat amb dades ambientals, com l'Índex de Qualitat de l'Aire (ICA), mitjançant agregacions espacials i temporals comunes.

El conjunt de dades ICA presenta una estructura clara i ben definida, amb informació espacial, temporal i ambiental adequada per a l'estudi de la qualitat de l'aire. Tot i això, la presència de valors nuls en variables clau com *indice* i *debido_a* fa necessària una fase de neteja i validació prèvia abans de l'anàlisi conjunta amb les dades de vols.

Amb els resultats de l'anàlisi exploratòria s'estableix el punt de partida per al següent bloc del treball, centrat en el preprocessament i la preparació de les dades per a l'anàlisi conjunta.